## M. Danilova, N. Puchkin, A. Shilova

## Large-scale optimization and applications course project

In [1]:
import numpy as np
import pandas as pd
import math
import time
import scipy as sc
import cvxpy as cvx

### Erdos-Renyi Random Graph

In [2]:
class ErdosRenyiGraph():
    def __init__(self, n=1, p=0.5):
        self.n = n # number of vertices
        self.p = p # edge occurrence probability
        self.A = None # adjacency matrix
        self.L = None # Laplacian matrix
        
    def generate_edges(self):
        # Generate edges
        R = np.random.rand(self.n, self.n)
        R = 1. * (R < self.p)
        R = np.triu(R)
        self.A = R + R.transpose()
        np.fill_diagonal(self.A, 0)
        
        # Compute diagonal
        D = np.sum(self.A, axis=1)
        # compute Laplacian
        self.L = np.diag(D) - self.A
        
        return

### Greedy algorithm

In [3]:
class Greedy():
    def __init__(self, get_x0=None, obj=None, tries=100):
        self.obj = obj
        self.get_x0 = get_x0 # function returning random x0
        self.x = None
        self.T = tries # number of tries
    
    def neighborhood(self, z):
    
        n = z.shape[0]
        neighbors = []

        for i in range(n):
            y = z[:]
            y[i] = -1 * z[i]
            neighbors += [y]

        return neighbors
    

    def greedy_search(self):
    
        obj_min = math.inf
        
        for i in range(self.T):
            self.x = self.get_x0()
            
            while (1):
                neighbors = self.neighborhood(self.x)
            
                obj_neighbors = np.array([self.obj(y) for y in neighbors])

                x_new = neighbors[np.argmin(obj_neighbors)]
                obj_min_new = self.obj(x_new)

                if obj_min_new < obj_min :
                    obj_min = obj_min_new
                    self.x = x_new
                else :
                    break

        return self.x, self.obj(self.x) 

In [4]:
def max_cut(x, A=None):
    n = A.shape[0]
    
    mc = 0.25 * (np.ones(n).transpose().dot(A).dot(np.ones(n)) - x.transpose().dot(A.dot(x)))
    return mc

In [5]:
def min_uncut(x, A=None):
    n = A.shape[0]
    
    muc = 0.25 * (np.ones(n).transpose().dot(A).dot(np.ones(n)) + x.transpose().dot(A.dot(x)))
    return muc

In [6]:
def sparsest_cut(x, A=None):
    n = A.shape[0]
    
    sc = (np.sum(A) - x.transpose().dot(A).dot(x)) / (n**2 - np.sum(x)**2)
    return sc

### Primal-Dual Central Path Tracing Interior Point Method

In [7]:
# Short-step primal-dual path tracing IPM solver for SDP
class IPM_Solver():
    def __init__(self, X0=None, S0=None,\
                 primal_projection=None,\
                 dual_projection=None,\
                 chi=0.1, kappa=0.1, n_iter=100):
        
        self.X = X0
        self.S = S0
        self.t = None
        self.primal_projection = primal_projection # replace this function for projection
        self.dual_projection = dual_projection # replace this function for projection
        self.chi = chi
        self.kappa = kappa
        self.n_iter = n_iter
        
        
    def calc_t(self):
        
        k = self.X.shape[0]
        self.t = k / np.trace(self.S.dot(self.X))
        
        return
        
        
    def check_initial(self):
        
        self.calc_t()
        sqrt_X = sc.linalg.sqrtm(self.X) 
        k = self.X.shape[0]
        norm = np.linalg.norm(self.t * sqrt_X.dot(self.S.dot(sqrt_X)) - np.identity(k), ord=2)
        
        violated = norm > self.kappa
        
        return violated
        
        
    def ipm_step(self):
        
        k = self.X.shape[0]
        X = self.X
        S = self.S
        self.calc_t()
        t1 = self.t / (1 - self.chi / np.sqrt(k))
        
        # Nesterov-Todd scaling matrix
        Q = NT_scaling_matrix(X, S)
        Q = Q.real
        #print(np.linalg.norm(sc.imag(Q)))
        Q_inv = np.linalg.inv(Q)
        
        X1 = Q.dot(X.dot(Q))
        S1 = Q_inv.dot(S.dot(Q_inv))
        B = 2 / t1 * np.identity(k) - 2 * X1.dot(X1)
        
        Z = sc.linalg.solve_lyapunov(X1, B)
        
        dS1 = self.dual_projection(Z, Q)
        dX1 = self.primal_projection(Z, Q)
        
        dX = Q_inv.dot(dX1.dot(Q_inv))
        dS = Q.dot(dS1.dot(Q))
        
        self.X = self.X + dX
        self.S = self.S + dS
        self.t = self.calc_t()
        
        return
        
    def solve(self):
        
        for i in range(self.n_iter):
            self.ipm_step()
        print('Iteration {}: duality gap = {}'.format(self.n_iter, self.duality_gap()))
        
    def duality_gap(self):
        return (np.trace(self.X.dot(self.S)))

In [8]:
def NT_scaling_matrix(X, S):
    
    #print('eigvals:', np.min(np.linalg.eigvals(X)), np.min(np.linalg.eigvals(S)))
    
    X12 = sc.linalg.sqrtm(X)
    X12_inv = np.linalg.pinv(X12)
    
    A = X12.dot(S.dot(X12))
    A12 = sc.linalg.sqrtm(A)
    A12_inv = np.linalg.pinv(A12)
    
    P = X12_inv.dot(A12_inv.dot(X12.dot(S)))
    
    Q = sc.linalg.sqrtm(P)
    
    return Q

In [9]:
def max_cut_dual_projection(Z, Q):
    
    k = Q.shape[0]
    
    Q_inv = np.linalg.inv(Q)
    
    basis = [np.outer(Q_inv[:, i], Q_inv[:, i]) for i in range(k)]
    basis = np.array([basis[i].reshape(-1) for i in range(k)])
    basis = basis.transpose()
    orth = sc.linalg.orth(basis)
    if (orth.shape[1] < k):
        print('Error: orth basis is bad:', orth.shape)
        return np.zeros((k, k))
    
    orth = [orth[:, i].reshape(k, k) for i in range(k)]
    c = np.array([np.trace(Z.dot(orth[i])) for i in range(k)])
    
    proj = np.zeros((k,k))
    for i in range(k):
        proj += c[i] * orth[i]
        
    return proj

In [10]:
def max_cut_primal_projection(Z, Q):
    
    return Z - max_cut_dual_projection(Z, Q)

In [179]:
def unit_l2_sp_cut(W):
    k = W.shape[0]
    X = cvx.Semidef(k)

    obj = cvx.Minimize(1. / k * cvx.trace(W * X))
    constraints = [cvx.diag(X) == 1, cvx.sum_entries(X) == k**2 - 0.5 * k - 1]
    for j in range(k):
        Ej = np.concatenate([np.zeros((j, k)), np.ones((1, k)), np.zeros((k - j - 1, k))])
        constraints += [X * Ej + Ej.T * X - X <= 1]

    prob = cvx.Problem(obj, constraints)

    prob.solve(solver='SCS', max_iters=100)
    print(prob.status)
    
    return prob.value

# Tests

### Tests on Erdos-Renyi  random graphs

### Parameters grid

In [11]:
params = {'n' : np.arange(20, 101, 20), 'p' : np.linspace(0.2, 0.8, 4)}

### Random graphs generating

In [12]:
# Auxiliary function
def get_key(n, p):
    return n + 10 * p

# Adjacency matrices of random graphs

graphs = {}
for n in params['n']:
    for p in params['p']:
        
        rg = ErdosRenyiGraph(n=n, p=p)
        rg.generate_edges()
        
        graphs[get_key(n, p)] = rg.A

### Theoretical estimation of Max-Cut, Min UnCut and Sparsest Cut values

In [158]:
# Theoretical estimation of max-cut expected value
def estimate_max_cut(n, p):
    
    # High-density max-cut estimation
    est1 = (0.25 * n * p + np.sqrt(n * p * np.log(2) / 4)) * n
    # Low-density max-cut estimation
    eps = 0.5 * n * p - 0.5
    est2 = (0.5 * eps + eps**3 / np.log(eps)) * n
    
    # Choose the tightest upper bound
    est = np.minimum(est1, est2)
    
    return est

# Theoretical estimation of min uncut expected value
def estimate_min_uncut(n, p):
    
    # High-density min uncut estimation
    est1 = 0.5 * n * (n - 1) * p - (0.25 * n * p + np.sqrt(4 * n * p / 9 / np.pi)) * n
    # Low-density min uncut estimation
    eps = 0.5 * n * p - 0.5
    est2 = 0.5 * n * (n - 1) * p - (0.5 * eps - 16 * eps**3 / 3) * n
    
    # Choose the tightest upper bound
    est = np.minimum(est1, est2)
    
    return est

# Theoretical estimation of sparsest cut lower bound via Cheeger inequality
def estimate_sparsest_cut(A):
    D = np.diag(np.sum(A, axis=1))
    L = D - A
    eigvals = np.linalg.eigvalsh(L)
    
    zero_eigvals = eigvals[eigvals < 1e-8]
    # If zero eigenvalue has a multiplicity 2,
    # then graph is not connected,
    # and the sparsest cut problem becomes trivial
    if zero_eigvals.shape[0] > 1:
        return 0
    
    eigvals = eigvals[eigvals > 1e-8]
    lambda_2 = np.min(eigvals)
    
    # Cheeger's upper bound
    return np.sqrt(2 * lambda_2)

In [156]:
max_cut_est_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
max_cut_est_values = pd.DataFrame(max_cut_est_values, index=params['p'], columns=params['n'])

min_uncut_est_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
min_uncut_est_values = pd.DataFrame(min_uncut_est_values, index=params['p'], columns=params['n'])

sp_cut_est_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sp_cut_est_values = pd.DataFrame(sp_cut_est_values, index=params['p'], columns=params['n'])

In [159]:
for n in params['n']:
    for p in params['p']:
        
        A = graphs[get_key(n, p)]
        max_cut_est_values.loc[p, n] = estimate_max_cut(n, p)
        min_uncut_est_values.loc[p, n] = estimate_min_uncut(n, p)
        sp_cut_est_values.loc[p, n] = estimate_sparsest_cut(A)

In [163]:
print(sp_cut_est_values)

          20        40        60         80         100
0.2  1.025329  2.124908  2.887202   3.502232   4.229240
0.4  2.338375  4.115762  5.009964   6.483546   7.343849
0.6  3.390212  4.963901  7.028810   8.478370   9.403919
0.8  5.020892  6.958765  8.597282  10.061558  11.374313


### Greedy algorithm test

In [85]:
# outputs of algorithm
mc_greedy_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
mc_greedy_values = pd.DataFrame(mc_greedy_values, index=params['p'], columns=params['n'])


# time of execution
mc_greedy_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
mc_greedy_times = pd.DataFrame(mc_greedy_times, index=params['p'], columns=params['n'])

In [89]:
# Max-Cut test 
for n in params['n']:
    for p in params['p']:

        gr = Greedy(get_x0=lambda : -1 * np.ones(n) + 2 * np.random.randint(2, size=n),\
                    obj=lambda x: -1 * max_cut(x, A=graphs[get_key(n, p)]))
        
        t = time.time()
        opt_x, opt_obj = gr.greedy_search()
        t = time.time() - t
        
        mc_greedy_values.loc[p, n] = -1 * opt_obj
        mc_greedy_times.loc[p, n] = t
        
print('Optimal values:\n', mc_greedy_values)
print('Estimation:\n', max_cut_est_values)
print('Time of execution:\n', mc_greedy_times)

Optimal values:
       20     40     60      80      100
0.2  17.0   91.0  165.0   302.0   512.0
0.4  27.0  163.0  372.0   623.0   994.0
0.6  68.0  233.0  523.0   976.0  1497.0
0.8  78.0  300.0  689.0  1250.0  1961.0
Estimation:
             20          40          60           80           100
0.2   35.045056  122.553843  258.176402   440.360444   668.208835
0.4   61.276922  220.180222  470.558128   810.215373  1237.883215
0.6   86.058801  313.705419  675.405500  1168.470405  1791.346248
0.8  110.090111  405.107686  876.352804  1520.720889  2336.417670
Time of execution:
           20        40        60        80        100
0.2  0.047625  0.106680  0.118408  0.179470  0.389374
0.4  0.059642  0.077786  0.122963  0.183532  0.353812
0.6  0.055438  0.073183  0.179925  0.290892  0.347230
0.8  0.052569  0.075561  0.218553  0.198785  0.264501


In [86]:
# outputs of algorithm
muc_greedy_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
muc_greedy_values = pd.DataFrame(mc_greedy_values, index=params['p'], columns=params['n'])


# time of execution
muc_greedy_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
muc_greedy_times = pd.DataFrame(mc_greedy_times, index=params['p'], columns=params['n'])

In [88]:
# Min UnCut test 
for n in params['n']:
    for p in params['p']:

        gr = Greedy(get_x0=lambda : -1 * np.ones(n) + 2 * np.random.randint(2, size=n),\
                    obj=lambda x: min_uncut(x, A=graphs[get_key(n, p)]))
        
        t = time.time()
        opt_x, opt_obj = gr.greedy_search()
        t = time.time() - t
        
        muc_greedy_values.loc[p, n] = opt_obj
        muc_greedy_times.loc[p, n] = t
        
print('Optimal values:\n', muc_greedy_values)
print('Estimation:\n', min_uncut_est_values)
print('Time of execution:\n', muc_greedy_times)

Optimal values:
       20     40     60      80      100
0.2  17.0   84.0  182.0   300.0   478.0
0.4  36.0  167.0  351.0   637.0  1048.0
0.6  65.0  236.0  549.0   944.0  1486.0
0.8  72.0  313.0  688.0  1245.0  1958.0
Estimation:
            20          40          60          80           100
0.2   1.348908   28.903599   87.478387  178.791262   303.835129
0.4  12.451800   85.395631  225.639961  435.614396   716.723115
0.6  25.159462  146.426641  372.140170  705.275698  1147.552986
0.8  38.697816  209.807198  522.956774  981.582524  1587.670259
Time of execution:
           20        40        60        80        100
0.2  0.055647  0.077939  0.128024  0.181162  0.428578
0.4  0.042858  0.075814  0.128913  0.244440  0.267759
0.6  0.036285  0.073086  0.132488  0.252940  0.312637
0.8  0.043720  0.071493  0.130774  0.198140  0.344343


In [90]:
# outputs of algorithm
sc_greedy_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sc_greedy_values = pd.DataFrame(mc_greedy_values, index=params['p'], columns=params['n'])


# time of execution
sc_greedy_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sc_greedy_times = pd.DataFrame(mc_greedy_times, index=params['p'], columns=params['n'])

In [91]:
# Sparsest Cut test 
for n in params['n']:
    for p in params['p']:

        gr = Greedy(get_x0=lambda : -1 * np.ones(n) + 2 * np.random.randint(2, size=n),\
                    obj=lambda x: sparsest_cut(x, A=graphs[get_key(n, p)]))
        
        t = time.time()
        opt_x, opt_obj = gr.greedy_search()
        t = time.time() - t
        
        sc_greedy_values.loc[p, n] = opt_obj
        sc_greedy_times.loc[p, n] = t
        
print('Optimal values:\n', sc_greedy_values)
print('Estimation:\n', sp_cut_est_values)
print('Time of execution:\n', sc_greedy_times)

Optimal values:
           20        40        60        80        100
0.2  0.190000  0.230000  0.208705  0.198730  0.200000
0.4  0.440000  0.393484  0.416295  0.386364  0.416967
0.6  0.703297  0.598465  0.595982  0.601001  0.601600
0.8  0.828283  0.804688  0.786756  0.774984  0.795918
Estimation:
           20         40         60         80         100
0.2  0.262825   1.128808   2.083984   3.066408   4.471618
0.4  1.366999   4.234874   6.274936  10.509092  13.483028
0.6  2.873385   6.160079  12.351044  17.970689  22.108422
0.8  6.302340  12.106101  18.478316  25.308740  32.343750
Time of execution:
           20        40        60        80        100
0.2  0.070913  0.083236  0.146062  0.385446  0.317414
0.4  0.057796  0.080928  0.140337  0.379452  0.446185
0.6  0.046991  0.092468  0.141134  0.294798  0.401081
0.8  0.043774  0.082179  0.151729  0.211405  0.349521


### Interior Point Method Test

In [21]:
# outputs of algorithm
ipm_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
ipm_values = pd.DataFrame(ipm_values, index=params['p'], columns=params['n'])


# time of execution
ipm_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
ipm_times = pd.DataFrame(ipm_times, index=params['p'], columns=params['n'])

In [174]:
# Max-Cut test 
for n in params['n']:
    for p in params['p']:

        W = graphs[get_key(n, p)]
    
        eigvals = np.linalg.eigvals(W)

        delta = 1

        lambda_max = np.max(eigvals) 
        alpha = 1 / (lambda_max + delta) 
        beta = delta - np.min(eigvals)
        
        X0 = np.identity(n) - alpha * W
        S0 = W + beta * np.identity(n)

        ipm_solver = IPM_Solver(X0=X0, S0=S0,\
                        primal_projection=max_cut_primal_projection,\
                        dual_projection=max_cut_dual_projection,\
                        n_iter=1000)

        t = time.time()
        ipm_solver.solve()
        t = time.time() - t
        
        X_opt = ipm_solver.X
        obj_opt = 0.25 * (np.sum(W) - np.trace(W.dot(X_opt)))
        
        ipm_values.loc[p, n] = obj_opt
        ipm_times.loc[p, n] = t
        
print('Optimal values:\n', ipm_values)
print('Estimation:\n', max_cut_est_values)
print('Time of execution:\n', ipm_times)

Iteration 1000: duality gap = 1.0919819125665942e-08
Iteration 1000: duality gap = 1.3641905032063698e-08
Iteration 1000: duality gap = 1.2488844262016414e-08
Iteration 1000: duality gap = 1.086633841143851e-08
Iteration 1000: duality gap = 2.541255360303508e-05
Iteration 1000: duality gap = 3.101375056731488e-05
Iteration 1000: duality gap = 2.9109624675103916e-05
Iteration 1000: duality gap = 2.580725896413288e-05
Iteration 1000: duality gap = 0.0008342496934483131
Iteration 1000: duality gap = 0.001052582359313518
Iteration 1000: duality gap = 0.0010290406674659078
Iteration 1000: duality gap = 0.0008545638193061183
Iteration 1000: duality gap = 0.00754825706357014
Iteration 1000: duality gap = 0.008952490886382098
Iteration 1000: duality gap = 0.008751721591362744
Iteration 1000: duality gap = 0.008005301876075243
Iteration 1000: duality gap = 0.034649555246937136
Iteration 1000: duality gap = 0.043398938685091225
Iteration 1000: duality gap = 0.042791081906036546
Iteration 1000: d

### Unit $l_2$ representation of sparsest cut

In [164]:
# outputs of algorithm
l2_sp_cut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
l2_sp_cut_values = pd.DataFrame(l2_sp_cut_values, index=params['p'], columns=params['n'])


# time of execution
l2_sp_cut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
l2_sp_cut_times = pd.DataFrame(l2_sp_cut_times, index=params['p'], columns=params['n'])

In [180]:
# Sparsest Cut test 
for n in params['n']:
    for p in params['p']:

        W = graphs[get_key(n, p)]
    
        t = time.time()
        obj_opt =  unit_l2_sp_cut(W)
        t = time.time() - t
        
        l2_sp_cut_values.loc[p, n] = obj_opt
        l2_sp_cut_times.loc[p, n] = t
        
        print('n =', n, ', p = ', p, 'tested')
        
        
print('Optimal values:\n', l2_sp_cut_values)
print('Estimation:\n', sp_cut_est_values)
print('Time of execution:\n', l2_sp_cut_times)

optimal_inaccurate
n = 20 , p =  0.2 tested
optimal_inaccurate
n = 20 , p =  0.4 tested
optimal_inaccurate
n = 20 , p =  0.6 tested
optimal_inaccurate
n = 20 , p =  0.8 tested
optimal_inaccurate
n = 40 , p =  0.2 tested
optimal_inaccurate
n = 40 , p =  0.4 tested
optimal_inaccurate
n = 40 , p =  0.6 tested
optimal_inaccurate
n = 40 , p =  0.8 tested
optimal_inaccurate
n = 60 , p =  0.2 tested
optimal_inaccurate
n = 60 , p =  0.4 tested
optimal_inaccurate
n = 60 , p =  0.6 tested
optimal_inaccurate
n = 60 , p =  0.8 tested
optimal_inaccurate
n = 80 , p =  0.2 tested
optimal_inaccurate
n = 80 , p =  0.4 tested
optimal_inaccurate
n = 80 , p =  0.6 tested
optimal_inaccurate
n = 80 , p =  0.8 tested
optimal_inaccurate
n = 100 , p =  0.2 tested
optimal_inaccurate
n = 100 , p =  0.4 tested
optimal_inaccurate
n = 100 , p =  0.6 tested
optimal_inaccurate
n = 100 , p =  0.8 tested
Optimal values:
            20         40         60         80         100
0.2   3.182472   8.727526  12.279557  15

# LP-relaxations

In [39]:
def Max_Cut_lp_relaxation(W):
    k = W.shape[0]
    
    x = cvx.Variable(k)
    Y = cvx.Variable(k, k)
    
    obj = cvx.Maximize(cvx.trace(W * Y))
    constraints = [Y <= x * np.ones((1, k)) + np.ones((k, 1)) * x.T,
                   Y <= 2 * np.ones((k, k)) - x * np.ones((1, k)) - np.ones((k, 1)) * x.T,
                   0 <= x, x <= 1, 
                   0 <= Y, Y <= 1]
    
    prob = cvx.Problem(obj, constraints)
    prob.solve(solver='SCS')
    
    return (np.ones(k).transpose().dot(W).dot(np.ones(k)) - x.value.transpose().dot(W.dot(x.value))) * 0.25

In [40]:
def transform_W(W):
    n = W.shape[0]
    W_new = np.zeros((2*n,2*n))
    
    W_new[n:, :n] = W
    W_new[:n, n:] = W
    
    return W_new

In [139]:
def Min_Un_Cut_lp_relaxation(W):
    k = W.shape[0]
    W = transform_W(W)
    
    x = cvx.Variable(2*k)
    Y = cvx.Variable(2*k, 2*k)
    
    obj = cvx.Minimize(cvx.trace(W * Y))
    constraints = [Y >= x * np.ones((1, 2*k)) + np.ones((2*k, 1)) * x.T,
                   Y >= 2 * np.ones((2*k, 2*k)) - x * np.ones((1, 2*k)) - np.ones((2*k, 1)) * x.T,
                   0 <= x, x <= 1,
                   Y.T == Y]
    
    prob = cvx.Problem(obj, constraints)
    prob.solve(solver='SCS')
    
    return (np.ones(2 * k).transpose().dot(W).dot(np.ones(2 * k)) + x.value.transpose().dot(W.dot(x.value))) * 0.125

In [169]:
def Sparsest_Cut_lp_relaxation(W):
    k = W.shape[0]
    
    Y = cvx.Variable(k, k)
    D = np.ones([k,k])
    
    obj = cvx.Maximize(cvx.trace(W * Y))
    constraints = [cvx.trace(D * Y) == 1, 
                   Y >= 0]
    for j in range(k):
        Ej = np.concatenate([np.zeros((j, k)), np.ones((1, k)), np.zeros((k - j - 1, k))])
        constraints += [Y * Ej + Ej.T * Y - Y >= 0]
            
    prob = cvx.Problem(obj, constraints)
    prob.solve(solver='SCS', max_iters=100)
    
    return prob.value

In [50]:
# outputs of algorithm
lp_Max_Cut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
lp_Max_Cut_values = pd.DataFrame(lp_Max_Cut_values, index=params['p'], columns=params['n'])


# time of execution
lp_Max_Cut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
lp_Max_Cut_times = pd.DataFrame(lp_Max_Cut_times, index=params['p'], columns=params['n'])

In [51]:
# outputs of algorithm
lp_Min_Uncut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
lp_Min_Uncut_values = pd.DataFrame(lp_Min_Uncut_values, index=params['p'], columns=params['n'])


# time of execution
lp_Min_Uncut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
lp_Min_Uncut_times = pd.DataFrame(lp_Min_Uncut_times, index=params['p'], columns=params['n'])

In [52]:
# outputs of algorithm
lp_sp_Cut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
lp_sp_Cut_values = pd.DataFrame(lp_sp_Cut_values, index=params['p'], columns=params['n'])


# time of execution
lp_sp_Cut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
lp_sp_Cut_times = pd.DataFrame(lp_sp_Cut_times, index=params['p'], columns=params['n'])

In [137]:
# Max-Cut test 
for n in params['n']:
    for p in params['p']:
        
        t = time.time()
        opt_obj = Max_Cut_lp_relaxation(graphs[get_key(n, p)])
        t = time.time() - t
        
        lp_Max_Cut_values.loc[p, n] = opt_obj
        lp_Max_Cut_times.loc[p, n] = t
        
print('Optimal values:\n', lp_Max_Cut_values)
print('Time of execution:\n', lp_Max_Cut_times)

Optimal values:
            20          40          60          80           100
0.2  12.374781   65.629918  138.386599  230.248402   370.509735
0.4  31.127516  113.624691  265.500843  471.378152   747.758466
0.6  47.629592  175.500947  397.500842  716.251030  1108.126786
0.8  60.003232  235.132637  525.013863  929.332810  1477.925121
Time of execution:
           20        40        60        80        100
0.2  0.013428  0.028338  0.047363  0.086294  0.148773
0.4  0.011856  0.056529  0.050932  0.125852  0.236400
0.6  0.012826  0.028367  0.071739  0.186027  0.354341
0.8  0.012313  0.032294  0.068754  0.134669  0.247148


In [141]:
# Min Un Cut test 
for n in params['n']:
    for p in params['p']:

#for n in [20]:
#    for p in [0.2]:

        t = time.time()
        opt_obj = Min_Un_Cut_lp_relaxation(graphs[get_key(n, p)])
        t = time.time() - t
        
        A = graphs[get_key(n, p)]
        lp_Min_Uncut_values.loc[p, n] = opt_obj
        lp_Min_Uncut_times.loc[p, n] = t

        print('n =', n, ', p = ', p, 'tested')
        
print('Optimal values:\n', lp_Min_Uncut_values)
print('Time of execution:\n', lp_Min_Uncut_times)

n = 20 , p =  0.2 tested
n = 20 , p =  0.4 tested
n = 20 , p =  0.6 tested
n = 20 , p =  0.8 tested
n = 40 , p =  0.2 tested
n = 40 , p =  0.4 tested
n = 40 , p =  0.6 tested
n = 40 , p =  0.8 tested
n = 60 , p =  0.2 tested
n = 60 , p =  0.4 tested
n = 60 , p =  0.6 tested
n = 60 , p =  0.8 tested
n = 80 , p =  0.2 tested
n = 80 , p =  0.4 tested
n = 80 , p =  0.6 tested
n = 80 , p =  0.8 tested
n = 100 , p =  0.2 tested
n = 100 , p =  0.4 tested
n = 100 , p =  0.6 tested
n = 100 , p =  0.8 tested
Optimal values:
            20          40          60           80           100
0.2  20.625516  109.373558  230.628054   383.755518   617.451226
0.4  51.876599  189.373887  442.495832   785.628966  1246.246914
0.6  79.375447  292.499432  662.477723  1193.733360  1846.903638
0.8  99.998919  391.877859  875.001162  1548.748177  2463.162666
Time of execution:
           20        40        60        80        100
0.2  0.029958  0.111946  0.347665  0.509658  0.765721
0.4  0.028274  0.113997  0

In [170]:
# Sparsest test 
for n in params['n']:
    for p in params['p']:
        
        t = time.time()
        opt_obj = Sparsest_Cut_lp_relaxation(graphs[get_key(n, p)])
        t = time.time() - t
        
        lp_sp_Cut_values.loc[p, n] = opt_obj
        lp_sp_Cut_times.loc[p, n] = t
        
        print('n =', n, ', p = ', p, 'tested')
        
print('Optimal values:\n', lp_sp_Cut_values)
print('Time of execution:\n', lp_sp_Cut_times)

n = 20 , p =  0.2 tested
n = 20 , p =  0.4 tested
n = 20 , p =  0.6 tested
n = 20 , p =  0.8 tested
n = 40 , p =  0.2 tested
n = 40 , p =  0.4 tested
n = 40 , p =  0.6 tested
n = 40 , p =  0.8 tested
n = 60 , p =  0.2 tested
n = 60 , p =  0.4 tested
n = 60 , p =  0.6 tested
n = 60 , p =  0.8 tested
n = 80 , p =  0.2 tested
n = 80 , p =  0.4 tested
n = 80 , p =  0.6 tested
n = 80 , p =  0.8 tested
n = 100 , p =  0.2 tested
n = 100 , p =  0.4 tested
n = 100 , p =  0.6 tested
n = 100 , p =  0.8 tested
Optimal values:
           20        40        60        80        100
0.2  0.298821  0.366473  0.345016  0.325384  0.332736
0.4  0.610984  0.559551  0.571442  0.569160  0.574307
0.6  0.827207  0.750154  0.749124  0.753448  0.747620
0.8  0.945677  0.901116  0.885715  0.879609  0.886595
Time of execution:
           20        40        60        80         100
0.2  0.117843  0.718436  2.739953  7.626631  12.672801
0.4  0.096559  0.749443  2.699008  7.180649  13.982816
0.6  0.096460  0.777362 

# Gradient descent for SDP-relaxations

In [75]:
def projection_1(X, mu, W):
    tmp = W - mu*(np.linalg.inv(X))
    for i in range(n):
        tmp[i,i] = 0;
    return tmp

In [76]:
def alpha(X, mu, W, beta):
    alpha_ = (beta * min(np.linalg.eig(X)[0])) / abs((min(np.linalg.eig(projection_1(X, mu, W))[0])))
    return alpha_

In [77]:
def GD_Max_Cut(W, l):
    n = W.shape[0]
    X_0 = np.eye(n)
    mu = 0.5
    beta = 0.5
    for i in range(l):
        X_0 = X_0 + (alpha(X_0, mu, W, beta))*projection_1(X_0, mu, W)
        mu = mu / 2
    return np.trace(W.dot(X_0))

In [78]:
def projection_2(X, mu, W):
    tmp = W - mu*(np.linalg.inv(X))
    for i in range(n):
        tmp[i,i] = 0
        tmp[i,n + i] = 0
        tmp[n + i, i] = 0
    return tmp

In [79]:
def alpha_2(X, mu, W, beta):
    alpha_ = (beta * min(np.linalg.eig(X)[0])) / abs((min(np.linalg.eig(projection_2(X, mu, W))[0])))
    return alpha_

In [80]:
def GD_Min_Un_Cut(W, l):
    W = transform_W(W)
    n = W.shape[0]
    X_0 = np.eye(n)
    mu = 0.5
    beta = 0.5
    for i in range(l):
        X_0 = X_0 + (alpha_2(X_0, mu, W, beta))*projection_2(X_0, mu, W)
        mu = mu / 2
    return np.trace(W.dot(X_0))

In [81]:
# outputs of algorithm
gd_Max_Cut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
gd_Max_Cut_values = pd.DataFrame(gd_Max_Cut_values, index=params['p'], columns=params['n'])


# time of execution
gd_Max_Cut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
gd_Max_Cut_times = pd.DataFrame(gd_Max_Cut_times, index=params['p'], columns=params['n'])

In [82]:
# Max-Cut test 
for n in params['n']:
    for p in params['p']:
        
        t = time.time()
        opt_obj = GD_Max_Cut(graphs[get_key(n, p)], 1000)
        t = time.time() - t
        
        W = graphs[get_key(n, p)]
        gd_Max_Cut_values.loc[p, n] = (W.sum() - opt_obj) * 0.25
        gd_Max_Cut_times.loc[p, n] = t
        
        print('n =', n, ', p = ', p, 'tested')
        
print('Optimal values:\n', gd_Max_Cut_values)
print('Time of execution:\n', gd_Max_Cut_times)

n = 20 , p =  0.2 tested
n = 20 , p =  0.4 tested
n = 20 , p =  0.6 tested
n = 20 , p =  0.8 tested
n = 40 , p =  0.2 tested
n = 40 , p =  0.4 tested
n = 40 , p =  0.6 tested
n = 40 , p =  0.8 tested
n = 60 , p =  0.2 tested
n = 60 , p =  0.4 tested
n = 60 , p =  0.6 tested
n = 60 , p =  0.8 tested
n = 80 , p =  0.2 tested
n = 80 , p =  0.4 tested
n = 80 , p =  0.6 tested
n = 80 , p =  0.8 tested
n = 100 , p =  0.2 tested
n = 100 , p =  0.4 tested
n = 100 , p =  0.6 tested
n = 100 , p =  0.8 tested
Optimal values:
            20          40          60           80           100
0.2  11.595010   70.732211  154.076731   264.365501   432.525958
0.4  32.152224  128.122607  308.316608   555.439156   898.486811
0.6  48.141263  195.779247  460.388988   841.820882  1329.801151
0.8  58.029009  256.156063  589.855538  1079.012900  1736.087154
Time of execution:
           20        40        60        80         100
0.2  0.415974  1.750458  3.857480  9.020489  15.484402
0.4  0.482390  1.643881 

In [83]:
# outputs of algorithm
gd_Min_Uncut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
gd_Min_Uncut_values = pd.DataFrame(gd_Min_Uncut_values, index=params['p'], columns=params['n'])


# time of execution
gd_Min_Uncut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
gd_Min_Uncut_times = pd.DataFrame(gd_Min_Uncut_times, index=params['p'], columns=params['n'])

In [142]:
# Min Un Cut test 
for n in params['n']:
    for p in params['p']:
        
        t = time.time()
        opt_obj = GD_Min_Un_Cut(graphs[get_key(n, p)], 1000)
        t = time.time() - t
        
        W = graphs[get_key(n, p)]
        gd_Min_Uncut_values.loc[p, n] = (W.sum() + 0.5 * opt_obj) * 0.25
        gd_Min_Uncut_times.loc[p, n] = t
        
        print('n =', n, ', p = ', p, 'tested')
        
print('Optimal values:\n', gd_Min_Uncut_values)
print('Time of execution:\n', gd_Min_Uncut_times)

n = 20 , p =  0.2 tested
n = 20 , p =  0.4 tested
n = 20 , p =  0.6 tested
n = 20 , p =  0.8 tested
n = 40 , p =  0.2 tested
n = 40 , p =  0.4 tested
n = 40 , p =  0.6 tested
n = 40 , p =  0.8 tested
n = 60 , p =  0.2 tested
n = 60 , p =  0.4 tested
n = 60 , p =  0.6 tested
n = 60 , p =  0.8 tested
n = 80 , p =  0.2 tested
n = 80 , p =  0.4 tested
n = 80 , p =  0.6 tested
n = 80 , p =  0.8 tested
n = 100 , p =  0.2 tested
n = 100 , p =  0.4 tested
n = 100 , p =  0.6 tested
n = 100 , p =  0.8 tested
Optimal values:
            20          40          60           80           100
0.2  20.480390   96.402211  198.108485   325.860153   517.697193
0.4  45.992434  161.016116  368.440385   648.039157  1021.470021
0.6  68.245788  243.713372  544.725786   974.745598  1502.220736
0.8  84.892674  323.356270  714.857894  1258.850306  1995.363747
Time of execution:
           20        40         60         80         100
0.2  1.825647  8.452745  23.353299  41.375080  78.893934
0.4  1.559878  8.446

# SDP-relaxations

In [109]:
def Max_Cut_relaxation(W):
    n = W.shape[0]
    X = cvx.Semidef(n)
    obj = cvx.Minimize(cvx.trace(W * X))
    constraints = [cvx.diag(X) == 1]
    prob = cvx.Problem(obj, constraints)
    prob.solve()
    return X.value, prob.value

In [110]:
def maxcut_with_triangle_constraints(W):
    n = W.shape[0]
    X = cvx.Semidef(n)
    obj = cvx.Minimize(cvx.trace(W * X))
    constraints = [cvx.diag(X) == 1]
    for j in range(n):
        E = np.zeros((n,n))
        E[j,:] += 1
        constraints.append(1- X*E - E.T * X >= -X)
        constraints.append(-X*E-E.T*X - X <= 1)
    prob = cvx.Problem(obj, constraints)
    prob.solve(solver='SCS', max_iters=100)
    print(prob.status)
    return X.value, prob.value

In [111]:
def weights_transform(W):
    n = W.shape[0]
    W_new = np.zeros((2*n,2*n))
    W_new[:n, n:] = W
    W_new[n:, :n] = W
    return W_new

In [124]:
def MinUnCut_relaxation(W):
    n = W.shape[0]//2
    X = cvx.Semidef(2*n)
    obj = cvx.Maximize(cvx.trace(W * X))
    constraints = [cvx.diag(X) == 1]
    for i in range(n):
        constraints.append(X[i, n + i] == -1)
    prob = cvx.Problem(obj, constraints)
    prob.solve()
    return X.value, prob.value

In [122]:
def minuncut_with_triangle_constraints(W):
    n = W.shape[0]//2
    X = cvx.Semidef(2*n)
    obj = cvx.Maximize(cvx.trace(W * X))
    constraints = [cvx.diag(X) == 1]
    for i in range(n):
        constraints.append(X[i, n + i] == -1)
    for j in range(2*n):
        E = np.zeros((2*n,2*n))
        E[j,:] += 1
        constraints.append(1- X*E - E.T * X >= -X)
        constraints.append(-X*E-E.T*X - X <= 1)
    prob = cvx.Problem(obj, constraints)
    prob.solve(solver='SCS', max_iters=100)
    return X.value, prob.value

In [153]:
def SparsestCut_relaxation(W):
    n = W.shape[0]
    X = cvx.Semidef(n)
    t = cvx.Variable()
    
    objective = cvx.Minimize(t*cvx.sum_entries(W) - cvx.trace(W*X))
    constraints =[ -cvx.sum_entries(X) +  t*n**2 >= 1, t>=1.0/n**2, t<= 1.0/(4*(n-1)), cvx.diag(X) == t]

    prob = cvx.Problem(objective, constraints)
    prob.solve(solver='SCS')

    return X.value, t.value, prob.value

In [145]:
def sparsest_cut_with_triangle_constraints(W):
    n = W.shape[0]
    X = cvx.Semidef(n)
    t = cvx.Variable()

    objective = cvx.Minimize(t*cvx.sum_entries(W) - cvx.trace(W*X))
    constraints =[ -cvx.sum_entries(X) +  t*n**2 >= 1, t>=4.0/n**2, t<= 1.0/(n-1), cvx.diag(X) == t]
    for j in range(n):
        E = np.zeros((n,n))
        E[j,:] += 1
        constraints.append(t- X*E - E.T * X >= -X)
        constraints.append(-X*E-E.T*X - X <= t)
    prob = cvx.Problem(objective, constraints)
    prob.solve()

    return X.value, t.value, prob.value

    A3 = np.zeros((2,2))
    A3[1,0] += 1
    objective = cvx.Minimize(t)
    constraints =[t*A0 + A1 + t2*A2 + t1*A3 >> 0, t2 == -cvx.sum_entries(X) + n**2, t1 == W.sum() - cvx.trace(W*X), 
                  cvx.diag(X) == 1, t2 >= n-1]
    for j in range(n):
        E = np.zeros((n,n))
        E[j,:] += 1
        constraints.append(1- X*E - E.T * X >= -X)
        constraints.append(-X*E-E.T*X - X <= 1)
    prob = cvx.Problem(objective, constraints)
    prob.solve(solver='SCS', max_iters=50)
    return X.value, prob.value

## Max-cut

In [116]:
# outputs of algorithm
maxcut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
maxcut_values = pd.DataFrame(maxcut_values, index=params['p'], columns=params['n'])


# time of execution
rel_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
rel_times = pd.DataFrame(rel_times, index=params['p'], columns=params['n'])

In [117]:
# outputs of algorithm
no_tr_maxcut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
no_tr_maxcut_values = pd.DataFrame(no_tr_maxcut_values, index=params['p'], columns=params['n'])


# time of execution
no_tr_rel_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
no_tr_rel_times = pd.DataFrame(no_tr_rel_times, index=params['p'], columns=params['n'])

In [118]:
#without triangle constraints
for n in params['n']:
    for p in params['p']:
        A = graphs[get_key(n,p)]
        
        t = time.time()
        opt_x, opt_obj = Max_Cut_relaxation(A)
        t = time.time() - t
        
        no_tr_maxcut_values.loc[p, n] = (A.sum()- opt_obj)/4
        no_tr_rel_times.loc[p, n] = t
        
print('Optimal values:\n', no_tr_maxcut_values)
print('Estimation:\n', max_cut_est_values)
print('Time of execution:\n', no_tr_rel_times)

Optimal values:
            20          40          60           80           100
0.2  29.429075  130.128812  262.233445   430.726080   669.101873
0.4  60.286282  203.670109  458.250110   782.844893  1214.615613
0.6  81.152204  288.500151  631.255816  1108.898055  1702.134931
0.8  95.359142  359.032522  787.564656  1376.713726  2159.430065
Estimation:
             20          40          60           80           100
0.2   35.045056  122.553843  258.176402   440.360444   668.208835
0.4   61.276922  220.180222  470.558128   810.215373  1237.883215
0.6   86.058801  313.705419  675.405500  1168.470405  1791.346248
0.8  110.090111  405.107686  876.352804  1520.720889  2336.417670
Time of execution:
           20        40        60        80        100
0.2  0.034280  0.101418  0.331691  0.535867  1.057492
0.4  0.032636  0.080624  0.260028  0.684516  1.370277
0.6  0.022795  0.113248  0.288867  0.911601  1.373014
0.8  0.036301  0.148022  0.370161  0.757674  1.972998


In [119]:
#with triangle constraints
for n in params['n']:
    for p in params['p']:
        A = graphs[get_key(n,p)]
        
        t = time.time()
        opt_x, opt_obj = maxcut_with_triangle_constraints(A)
        t = time.time() - t
        
        maxcut_values.loc[p, n] = (A.sum()- opt_obj)/4
        rel_times.loc[p, n] = t
        
print('Optimal values:\n', maxcut_values)
print('Estimation:\n', max_cut_est_values)
print('Time of execution:\n', rel_times)

optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
optimal_inaccurate
Optimal values:
            20          40          60           80           100
0.2  27.891607  124.438259  253.708823   420.792165   667.741173
0.4  58.218397  197.525876  443.439515   780.335853  1214.781083
0.6  78.756198  278.534561  632.140892  1149.005359  1730.524935
0.8  93.401422  330.500775  807.900442  1528.203814  2388.861825
Estimation:
             20          40          60           80           100
0.2   35.045056  122.553843  258.176402   440.360444   668.208835
0.4   61.276922  220.180222  470.558128   810.215373  1237.883215
0.6   86.058801  313.705419  675.405500  1168.470405  1791.346248
0

## Min UnCut

In [125]:
# outputs of algorithm
minuncut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
minuncut_values = pd.DataFrame(minuncut_values, index=params['p'], columns=params['n'])

# time of execution
minuncut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
minuncut_times = pd.DataFrame(minuncut_times, index=params['p'], columns=params['n'])

In [126]:
#without triangle constraints
for n in params['n']:
    for p in params['p']:
        A = graphs[get_key(n,p)]
        A_transformed = weights_transform(A)
        
        t = time.time()
        opt_x, opt_obj = MinUnCut_relaxation(A_transformed)
        t = time.time() - t
        
        minuncut_values.loc[p, n] =(2 * A.sum() - opt_obj)/8
        minuncut_times.loc[p, n] = t
        
print('Optimal values:\n', minuncut_values)
print('Estimation:\n', min_uncut_est_values)
print('Time of execution:\n', minuncut_times)

Optimal values:
            20          40          60           80           100
0.2   3.571560   44.872762  106.772603   183.276765   318.907573
0.4  22.715442   99.328873  249.748394   474.168385   779.390037
0.6  45.848881  179.504169  428.746990   801.123661  1252.886558
0.8  64.642342  267.969947  612.445216  1101.298644  1781.601798
Estimation:
            20          40          60          80           100
0.2   1.348908   28.903599   87.478387  178.791262   303.835129
0.4  12.451800   85.395631  225.639961  435.614396   716.723115
0.6  25.159462  146.426641  372.140170  705.275698  1147.552986
0.8  38.697816  209.807198  522.956774  981.582524  1587.670259
Time of execution:
           20        40        60         80         100
0.2  0.276537  1.874202  6.227408  15.865747  31.877150
0.4  0.238374  2.002168  5.987302  16.097594  34.618038
0.6  0.313747  1.640704  7.582998  16.550395  31.338326
0.8  0.244984  1.833730  6.129244  15.752917  37.577764


In [127]:
# outputs of algorithm
tr_minuncut_values = np.zeros((params['p'].shape[0], params['n'].shape[0]))
tr_minuncut_values = pd.DataFrame(minuncut_values, index=params['p'], columns=params['n'])

# time of execution
tr_minuncut_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
tr_minuncut_times = pd.DataFrame(minuncut_times, index=params['p'], columns=params['n'])

In [185]:
#with triangle constraints
for n in params['n']:
    for p in params['p']:
        A = graphs[get_key(n,p)]
        A_transformed = weights_transform(A)
        t = time.time()
        opt_x, opt_obj = minuncut_with_triangle_constraints(A_transformed)
        t = time.time() - t
        
        tr_minuncut_values.loc[p, n] = (2 * A.sum() - opt_obj)/8
        tr_minuncut_times.loc[p, n] = t
        
        print('n =', n, 'p =', p, 'tested')
        
print('Optimal values:\n', tr_minuncut_values)
print('Estimation:\n', min_uncut_est_values)
print('Time of execution:\n', rel_times)

n = 20 p = 0.2 tested
n = 20 p = 0.4 tested
n = 20 p = 0.6 tested
n = 20 p = 0.8 tested
n = 40 p = 0.2 tested
n = 40 p = 0.4 tested
n = 40 p = 0.6 tested
n = 40 p = 0.8 tested
n = 60 p = 0.2 tested
n = 60 p = 0.4 tested
n = 60 p = 0.6 tested
n = 60 p = 0.8 tested
n = 80 p = 0.2 tested
n = 80 p = 0.4 tested
n = 80 p = 0.6 tested
n = 80 p = 0.8 tested
n = 100 p = 0.2 tested
n = 100 p = 0.4 tested
n = 100 p = 0.6 tested
n = 100 p = 0.8 tested
Optimal values:
            20         40         60          80          100
0.2   5.311028  32.291586  60.821678  111.420058  182.991256
0.4  25.769137  24.955998  25.004866   90.698746  196.782995
0.6  43.206710  34.119193  -0.291784   -0.489180   -0.476950
0.8  54.575761  45.616032  -1.438780   15.638582  157.723956
Estimation:
            20          40          60           80           100
0.2   2.954944   33.446157   95.823598   191.639556   321.791165
0.4  14.723078   91.819778  237.441872   453.784627   742.116785
0.6  27.941199  154.294581

## Sparsest cut

In [176]:
# outputs of algorithm
sdp_spcut_values_no_triangle = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sdp_spcut_values_no_triangle = pd.DataFrame(sdp_spcut_values_no_triangle, index=params['p'], columns=params['n'])
#with triangle constraints
sdp_spcut_values_with_triangle = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sdp_spcut_values_with_triangle = pd.DataFrame(sdp_spcut_values_with_triangle, index=params['p'], columns=params['n'])


# time of execution
sdp_spc_no_triangle_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sdp_spc_no_triangle_times = pd.DataFrame(sdp_spc_no_triangle_times, index=params['p'], columns=params['n'])

sdp_spc_with_triangle_times = np.zeros((params['p'].shape[0], params['n'].shape[0]))
sdp_spc_with_triangle_times = pd.DataFrame(sdp_spc_with_triangle_times, index=params['p'], columns=params['n'])

In [177]:
#without triangle constraints
for n in params['n']:
    for p in params['p']:
        A = graphs[get_key(n,p)]
        
        t = time.time()
        opt_x, opt_t, opt_obj = SparsestCut_relaxation(A)
        t = time.time() - t
        
        sdp_spcut_values_no_triangle.loc[p, n] =  opt_obj
        sdp_spc_no_triangle_times.loc[p, n] = t
        
        print('n =', n, 'p =', p, 'tested')
        
print('Optimal values:\n', sdp_spcut_values_no_triangle)
print('Estimation:\n', sp_cut_est_values)
print('Time of execution:\n', sdp_spc_no_triangle_times)

n = 20 p = 0.2 tested
n = 20 p = 0.4 tested
n = 20 p = 0.6 tested
n = 20 p = 0.8 tested
n = 40 p = 0.2 tested
n = 40 p = 0.4 tested
n = 40 p = 0.6 tested
n = 40 p = 0.8 tested
n = 60 p = 0.2 tested
n = 60 p = 0.4 tested
n = 60 p = 0.6 tested
n = 60 p = 0.8 tested
n = 80 p = 0.2 tested
n = 80 p = 0.4 tested
n = 80 p = 0.6 tested
n = 80 p = 0.8 tested
n = 100 p = 0.2 tested
n = 100 p = 0.4 tested
n = 100 p = 0.6 tested
n = 100 p = 0.8 tested
Optimal values:
           20        40        60        80        100
0.2  0.028951  0.060978  0.074325  0.081575  0.095858
0.4  0.143840  0.216840  0.214164  0.269137  0.277123
0.6  0.295262  0.314169  0.421538  0.458599  0.456832
0.8  0.637243  0.612900  0.626172  0.647002  0.662418
Estimation:
           20        40        60         80         100
0.2  1.025329  2.124908  2.887202   3.502232   4.229240
0.4  2.338375  4.115762  5.009964   6.483546   7.343849
0.6  3.390212  4.963901  7.028810   8.478370   9.403919
0.8  5.020892  6.958765  8.59728

In [152]:
#with triangle constraints

# WORKED VERY SLOWLY !!!
for n in params['n']:
    for p in params['p']:
        A = graphs[get_key(n,p)]
        
        t = time.time()
        opt_x, opt_t, opt_obj = sparsest_cut_with_triangle_constraints(A)
        t = time.time() - t
        
        sdp_spcut_values_with_triangle.loc[p, n] =  opt_obj
        sdp_spc_with_triangle_times.loc[p, n] = t
        
        print('n =', n, 'p =', p, 'tested')
        
        
print('Optimal values:\n', sdp_spcut_values_with_triangle)
print('Estimation:\n', sp_cut_est_values)
print('Time of execution:\n', sdp_spc_with_triangle_times)

n = 20 p = 0.2 tested
n = 20 p = 0.4 tested
n = 20 p = 0.6 tested
n = 20 p = 0.8 tested
n = 40 p = 0.2 tested
n = 40 p = 0.4 tested
n = 40 p = 0.6 tested
n = 40 p = 0.8 tested
n = 60 p = 0.2 tested
n = 60 p = 0.4 tested
n = 60 p = 0.6 tested
n = 60 p = 0.8 tested
n = 80 p = 0.2 tested
Failure:Interrupted


KeyError: 'Interrupted'

# Alena's results

In [186]:
print('SDP Max-Cut')
print('without triangle constraints')
sdp_max_cut_quality = maxcut_values
sdp_max_cut_times = rel_times
print(sdp_max_cut_quality.to_latex())
print(sdp_max_cut_times.to_latex())

print('with triangle constraints')
sdp_no_tr_max_cut_quality = no_tr_maxcut_values
sdp_no_tr_max_cut_times = no_tr_rel_times
print(sdp_no_tr_max_cut_quality.to_latex())
print(sdp_no_tr_max_cut_times.to_latex())

print('SDP Min Uncut')
print('without triangle constraints')
sdp_min_uncut_quality = minuncut_values
sdp_min_uncut_times = minuncut_times
print(sdp_min_uncut_quality.to_latex())
print(sdp_min_uncut_times.to_latex())
      
      
print('with triangle constraints')
sdp_tr_min_uncut_quality = tr_minuncut_values
sdp_tr_min_uncut_times = tr_minuncut_times
print(sdp_tr_min_uncut_quality.to_latex())
print(sdp_tr_min_uncut_times.to_latex())

print('SDP Sparsest Cut')
print('without triangle constraints')
sdp_sp_cut_quality = sdp_spcut_values_no_triangle
sdp_sp_cut_times = sdp_spc_no_triangle_times
print(sdp_sp_cut_quality.to_latex())
print(sdp_sp_cut_times.to_latex())

SDP Max-Cut
without triangle constraints
\begin{tabular}{lrrrrr}
\toprule
{} &        20  &         40  &         60  &          80  &          100 \\
\midrule
0.2 &  27.891607 &  124.438259 &  253.708823 &   420.792165 &   667.741173 \\
0.4 &  58.218397 &  197.525876 &  443.439515 &   780.335853 &  1214.781083 \\
0.6 &  78.756198 &  278.534561 &  632.140892 &  1149.005359 &  1730.524935 \\
0.8 &  93.401422 &  330.500775 &  807.900442 &  1528.203814 &  2388.861825 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &       20  &       40  &       60  &        80  &        100 \\
\midrule
0.2 &  0.276537 &  1.874202 &  6.227408 &  15.865747 &  31.877150 \\
0.4 &  0.238374 &  2.002168 &  5.987302 &  16.097594 &  34.618038 \\
0.6 &  0.313747 &  1.640704 &  7.582998 &  16.550395 &  31.338326 \\
0.8 &  0.244984 &  1.833730 &  6.129244 &  15.752917 &  37.577764 \\
\bottomrule
\end{tabular}

with triangle constraints
\begin{tabular}{lrrrrr}
\toprule
{} &        20  &         40 

# Marina's results

In [183]:
print('LP Max-Cut')
lp_max_cut_quality = lp_Max_Cut_values
lp_max_cut_times = lp_Max_Cut_times
print(lp_max_cut_quality.to_latex())
print(lp_max_cut_times.to_latex())

print('LP Min Uncut')
lp_min_uncut_quality = lp_Min_Uncut_values
lp_min_uncut_times = lp_Min_Uncut_times
print(lp_min_uncut_quality.to_latex())
print(lp_min_uncut_times.to_latex())
      
print('LP Sparsest Cut')
lp_sp_cut_quality = lp_sp_Cut_values
lp_sp_cut_times = lp_sp_Cut_times
print(lp_sp_cut_quality.to_latex())
print(lp_sp_cut_times.to_latex())

print('Gradient descent results')
print('GD Max-Cut')
gd_max_cut_quality = gd_Max_Cut_values
gd_max_cut_times = gd_Max_Cut_times
print(gd_max_cut_quality.to_latex())
print(gd_max_cut_times.to_latex())

print('GD Min Uncut')
gd_min_uncut_quality = gd_Min_Uncut_values
gd_min_uncut_times = gd_Min_Uncut_times
print(gd_min_uncut_quality.to_latex())
print(gd_min_uncut_times.to_latex())


LP Max-Cut
\begin{tabular}{lrrrrr}
\toprule
{} &        20  &         40  &         60  &         80  &          100 \\
\midrule
0.2 &  12.374781 &   65.629918 &  138.386599 &  230.248402 &   370.509735 \\
0.4 &  31.127516 &  113.624691 &  265.500843 &  471.378152 &   747.758466 \\
0.6 &  47.629592 &  175.500947 &  397.500842 &  716.251030 &  1108.126786 \\
0.8 &  60.003232 &  235.132637 &  525.013863 &  929.332810 &  1477.925121 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &       20  &       40  &       60  &       80  &       100 \\
\midrule
0.2 &  0.013428 &  0.028338 &  0.047363 &  0.086294 &  0.148773 \\
0.4 &  0.011856 &  0.056529 &  0.050932 &  0.125852 &  0.236400 \\
0.6 &  0.012826 &  0.028367 &  0.071739 &  0.186027 &  0.354341 \\
0.8 &  0.012313 &  0.032294 &  0.068754 &  0.134669 &  0.247148 \\
\bottomrule
\end{tabular}

LP Min Uncut
\begin{tabular}{lrrrrr}
\toprule
{} &        20  &         40  &         60  &          80  &          100 \\
\midrule
0

## Nikita's results

In [184]:
print('Greedy algorithm')
print('Max-Cut')
gr_max_cut_quality = mc_greedy_values
gr_max_cut_times = mc_greedy_times
print(gr_max_cut_quality.to_latex())
print(gr_max_cut_times.to_latex())

print('Min Uncut')
gr_min_uncut_quality = muc_greedy_values
gr_min_uncut_times = muc_greedy_times
print(gr_min_uncut_quality.to_latex())
print(gr_min_uncut_times.to_latex())
      
print('Sparsest Cut')
gr_sp_cut_quality = sc_greedy_values
gr_sp_cut_times = sc_greedy_times
print(gr_sp_cut_quality.to_latex())
print(gr_sp_cut_times.to_latex())


print('IPM Max-Cut')
ipm_max_cut_quality = ipm_values
ipm_max_cut_times = ipm_times
print(ipm_max_cut_quality.to_latex())
print(ipm_max_cut_times.to_latex())

print('l2 Sparsest Cut')
l2_sp_cut_quality = l2_sp_cut_values
l2_sp_cut_times = l2_sp_cut_times
print(l2_sp_cut_quality.to_latex())
print(l2_sp_cut_times.to_latex())

Greedy algorithm
Max-Cut
\begin{tabular}{lrrrrr}
\toprule
{} &       20  &       40  &       60  &       80  &       100 \\
\midrule
0.2 &  0.190000 &  0.230000 &  0.208705 &  0.198730 &  0.200000 \\
0.4 &  0.440000 &  0.393484 &  0.416295 &  0.386364 &  0.416967 \\
0.6 &  0.703297 &  0.598465 &  0.595982 &  0.601001 &  0.601600 \\
0.8 &  0.828283 &  0.804688 &  0.786756 &  0.774984 &  0.795918 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &       20  &       40  &       60  &       80  &       100 \\
\midrule
0.2 &  0.070913 &  0.083236 &  0.146062 &  0.385446 &  0.317414 \\
0.4 &  0.057796 &  0.080928 &  0.140337 &  0.379452 &  0.446185 \\
0.6 &  0.046991 &  0.092468 &  0.141134 &  0.294798 &  0.401081 \\
0.8 &  0.043774 &  0.082179 &  0.151729 &  0.211405 &  0.349521 \\
\bottomrule
\end{tabular}

Min Uncut
\begin{tabular}{lrrrrr}
\toprule
{} &       20  &       40  &       60  &       80  &       100 \\
\midrule
0.2 &  0.190000 &  0.230000 &  0.208705 &  0.198730